In [1]:
!pip install -r requirement.txt

In [2]:
print("requirements done")

requirements done


In [3]:
import os

from dotenv import load_dotenv
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Chroma
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
# Load environment variables from .env
load_dotenv(".env")
# print(os.getenv("GOOGLE_API_KEY"))



/Users/pratham/miniconda3/envs/langchainenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
# current_dir = 
file_path = "./cleaned_txt/Wheat-Growers-Guide_cleaned.txt"
db_dir = os.path.join("./", "db")

In [5]:
with open(file_path,"r") as f:
    text=f.read()
    print(text[:500])
    print(len(text))

wheat growers guide contents soils and climate 2 varietal choice 2 land preparation and soil conditioning 2 soil conditioning 3 tillage procedures 3 time of planting 3 seeding rates 4 irrigation requirements and scheduling 4 fertilisation 5 weed control 6 pests and diseases 6 whe at production general tips 7 centre pivot irrigation scheduling a general guide 9 chemigationfertigation calibrations guide 9 harvesting 10 2 soils and climate wheat is a temperate crop and is best grown in winter under
18652


In [6]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter


# Check if the text file exists
if not os.path.exists(file_path):
    raise FileNotFoundError(
        f"The file {file_path} does not exist. Please check the path."
    )

# Read the text content from the file
loader = TextLoader(file_path)
documents = loader.load()

# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Display information about the split documents
print("\n--- Document Chunks Information ---")
print(f"Number of document chunks: {len(docs)}")
print(f"Sample chunk:\n{docs[0].page_content}\n")


# Function to create and persist vector store
def create_vector_store(docs, embeddings, store_name):
    persistent_directory = os.path.join(db_dir, store_name)
    if not os.path.exists(persistent_directory):
        print(f"\n--- Creating vector store {store_name} ---")
        Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory)
        print(f"--- Finished creating vector store {store_name} ---")
    else:
        print(
            f"Vector store {store_name} already exists. No need to initialize.")


--- Document Chunks Information ---
Number of document chunks: 1
Sample chunk:
wheat growers guide contents soils and climate 2 varietal choice 2 land preparation and soil conditioning 2 soil conditioning 3 tillage procedures 3 time of planting 3 seeding rates 4 irrigation requirements and scheduling 4 fertilisation 5 weed control 6 pests and diseases 6 whe at production general tips 7 centre pivot irrigation scheduling a general guide 9 chemigationfertigation calibrations guide 9 harvesting 10 2 soils and climate wheat is a temperate crop and is best grown in winter under irrigation with optimum day temperatures of between 15 20oc and cooler nights giving the best yields there are some varieties that may be grown in summer such as sahai but generally there is high disea se and weed pressure in summer accompanied by warmer temperatures that result in depressed yields 3tha therefore winter is the best time for growing wheat the crop is adapted to a wide range of soils the soils must be

In [9]:
import pickle
print("\n--- Using Hugging Face Transformers ---")
# huggingface_embeddings = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-mpnet-base-v2"
# )
huggingface_embeddings= pickle.load(open("huggingface_embeddings.pkl","rb"))
create_vector_store(docs, huggingface_embeddings, "chroma_db_huggingface")

print("Embedding demonstrations for OpenAI and Hugging Face completed.")



--- Using Hugging Face Transformers ---
Vector store chroma_db_huggingface already exists. No need to initialize.
Embedding demonstrations for OpenAI and Hugging Face completed.


In [10]:
def query_vector_store(store_name, query, embedding_function):
    persistent_directory = os.path.join(db_dir, store_name)
    if os.path.exists(persistent_directory):
        print(f"\n--- Querying the Vector Store {store_name} ---")
        db = Chroma(
            persist_directory=persistent_directory,
            embedding_function=embedding_function,
        )
        retriever = db.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={"k": 5, "score_threshold": 0.1},
        )
        relevant_docs = retriever.invoke(query)
        # Display the relevant results with metadata
        print(f"\n--- Relevant Documents for {store_name} ---")
        for i, doc in enumerate(relevant_docs, 1):
            print(f"Document {i}:\n{doc.page_content}\n")
            if doc.metadata:
                print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")
    else:
        print(f"Vector store {store_name} does not exist.")


In [11]:
# query="wheat is good in which soil?"
query="what pests can attack wheat?"

query_vector_store("chroma_db_huggingface", query, huggingface_embeddings)


--- Querying the Vector Store chroma_db_huggingface ---


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1



--- Relevant Documents for chroma_db_huggingface ---
Document 1:
wheat growers guide contents soils and climate 2 varietal choice 2 land preparation and soil conditioning 2 soil conditioning 3 tillage procedures 3 time of planting 3 seeding rates 4 irrigation requirements and scheduling 4 fertilisation 5 weed control 6 pests and diseases 6 whe at production general tips 7 centre pivot irrigation scheduling a general guide 9 chemigationfertigation calibrations guide 9 harvesting 10 2 soils and climate wheat is a temperate crop and is best grown in winter under irrigation with optimum day temperatures of between 15 20oc and cooler nights giving the best yields there are some varieties that may be grown in summer such as sahai but generally there is high disea se and weed pressure in summer accompanied by warmer temperatures that result in depressed yields 3tha therefore winter is the best time for growing wheat the crop is adapted to a wide range of soils the soils must be well drained 

In [12]:
import os

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Define the directory containing the text files and the persistent directory

books_dir = './cleaned_txt'
db_dir = os.path.join('./', "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata")

print(f"Books directory: {books_dir}")
print(f"Persistent directory: {persistent_directory}")

def create_vector_database(embeddings,persistent_directory):
    # Check if the Chroma vector store already exists
    if not os.path.exists(persistent_directory):
        print("Persistent directory does not exist. Initializing vector store...")

        # Ensure the books directory exists
        if not os.path.exists(books_dir):
            raise FileNotFoundError(
                f"The directory {books_dir} does not exist. Please check the path."
            )

        # List all text files in the directory
        book_files = [f for f in os.listdir(books_dir) if f.endswith(".txt")]

        # Read the text content from each file and store it with metadata
        documents = []
        for book_file in book_files:
            file_path = os.path.join(books_dir, book_file)
            loader = TextLoader(file_path)
            book_docs = loader.load()
            for doc in book_docs:
                # Add metadata to each document indicating its source
                doc.metadata = {"source": book_file}
                documents.append(doc)

        # Split the documents into chunks
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        docs = text_splitter.split_documents(documents)

        # Display information about the split documents
        print("\n--- Document Chunks Information ---")
        print(f"Number of document chunks: {len(docs)}")

        # Create embeddings
        print("\n--- Creating embeddings ---")
        # Update to a valid embedding model if needed
        print("\n--- Finished creating embeddings ---")

        # Create the vector store and persist it
        print("\n--- Creating and persisting vector store ---")
        db = Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory)
        print("\n--- Finished creating and persisting vector store ---")

    else:
        print("Vector store already exists. No need to initialize.")


Books directory: ./cleaned_txt
Persistent directory: ./db/chroma_db_with_metadata


In [13]:
# Save the object to a file
import pickle
with open('huggingface_embeddings.pkl', 'wb') as f:
    pickle.dump(huggingface_embeddings, f)

In [14]:
huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
create_vector_database(huggingface_embeddings,persistent_directory)

Vector store already exists. No need to initialize.


In [20]:
# Define the persistent directory
# current_dir = os.path.dirname(os.path.abspath(__file__))
# persistent_directory = os.path.join(current_dir, "db", "chroma_db_with_metadata")
persistent_directory = "./db/chroma_db_huggingface"

# Define the embedding model
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory, embedding_function=huggingface_embeddings)

# Create a retriever for querying the vector store
# `search_type` specifies the type of search (e.g., similarity)
# `search_kwargs` contains additional arguments for the search (e.g., number of results to return)
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

# Create a google model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Contextualize question prompt
# This system prompt helps the AI understand that it should reformulate the question
# based on the chat history to make it a standalone question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

# Create a prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a history-aware retriever
# This uses the LLM to help reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Answer question prompt
# This system prompt helps the AI understand that it should provide concise answers
# based on the retrieved context and indicates what to do if the answer is unknown
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use optimal number of sentences to answer the question. "
    "\n\n"
    "{context}"
)

# Create a prompt template for answering questions
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a chain to combine documents for question answering
# `create_stuff_documents_chain` feeds all retrieved context into the LLM
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create a retrieval chain that combines the history-aware retriever and the question answering chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# Function to simulate a continual chat
def continual_chat():
    print("Start chatting with the AI! Type 'exit' to end the conversation.")
    chat_history = []  # Collect chat history here (a sequence of messages)
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            print("Bye....")
            break
        # Process the user's query through the retrieval chain
        result = rag_chain.invoke({"input": query, "chat_history": chat_history})
        # Display the AI's response
        print(f"AI: {result['answer']}")
        # Update the chat history
#         chat_history.append(HumanMessage(content=query))
#         chat_history.append(SystemMessage(content=result["answer"]))
        chat_history.append(HumanMessage(content=query))
        chat_history.append(AIMessage(content=result["answer"]))
#         print(chat_history)


# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Start chatting with the AI! Type 'exit' to end the conversation.


You:  what is a fertilizer?


AI: A fertilizer is a substance that is added to soil to provide nutrients for plants. 



You:  give me a big answer.


AI: Fertilizers are substances that are added to soil to provide essential nutrients for plant growth. They are crucial for boosting crop yields and ensuring the health and vitality of plants. 

Fertilizers can be broadly categorized into two main types:

**1. Organic Fertilizers:** These are derived from natural sources, such as animal manure, compost, and plant-based materials. They are generally slow-releasing, providing nutrients gradually over time, and contribute to improving soil structure and microbial activity. 

**2. Inorganic Fertilizers:** These are manufactured synthetically and contain specific ratios of essential nutrients, primarily nitrogen (N), phosphorus (P), and potassium (K), often referred to as NPK. They are readily available and provide a quick and concentrated source of nutrients. 

**Here's a detailed breakdown of the key aspects of fertilizers:**

* **Essential Nutrients:** Fertilizers provide essential nutrients that plants need for healthy growth and develo

You:  exit


Bye....


In [15]:
# Define the persistent directory
# current_dir = os.path.dirname(os.path.abspath(__file__))
# persistent_directory = os.path.join(current_dir, "db", "chroma_db_with_metadata")
persistent_directory = ".db/chroma_db_with_metadata"

# Define the embedding model
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory, embedding_function=huggingface_embeddings)

# Create a retriever for querying the vector store
# `search_type` specifies the type of search (e.g., similarity)
# `search_kwargs` contains additional arguments for the search (e.g., number of results to return)
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

# Create a google model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Contextualize question prompt
# This system prompt helps the AI understand that it should reformulate the question
# based on the chat history to make it a standalone question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

# Create a prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a history-aware retriever
# This uses the LLM to help reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Answer question prompt
# This system prompt helps the AI understand that it should provide concise answers
# based on the retrieved context and indicates what to do if the answer is unknown
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use optimal number of sentences to answer the question. "
    "Provide the source as well. "
    "\n\n"
    "{context}"
)

# Create a prompt template for answering questions
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a chain to combine documents for question answering
# `create_stuff_documents_chain` feeds all retrieved context into the LLM
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create a retrieval chain that combines the history-aware retriever and the question answering chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# Function to simulate a continual chat
def continual_chat():
    print("Start chatting with the AI! Type 'exit' to end the conversation.")
    chat_history = []  # Collect chat history here (a sequence of messages)
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            print("Bye....")
            break
        # Process the user's query through the retrieval chain
        result = rag_chain.invoke({"input": query, "chat_history": chat_history})
        # Display the AI's response
        print(f"AI: {result['answer']}")
        # Update the chat history
#         chat_history.append(HumanMessage(content=query))
#         chat_history.append(SystemMessage(content=result["answer"]))
        chat_history.append(HumanMessage(content=query))
        chat_history.append(AIMessage(content=result["answer"]))
#         print(chat_history)


# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()


Start chatting with the AI! Type 'exit' to end the conversation.
AI: Hello! 👋 How can I help you today? 😊 

AI: It's not possible to grow wheat and rice on the same land simultaneously. Both crops require different growing conditions and planting times. 

However, you can **rotate** wheat and rice crops on the same land. This practice, called crop rotation, offers several benefits:

* **Nutrient management:** Wheat and rice have different nutrient requirements. Rotating them helps maintain soil fertility by using up different nutrients in the soil.
* **Pest and disease control:** Rotating crops can help break the life cycles of pests and diseases specific to one crop, reducing their impact.
* **Improved soil health:** Different crops have different root structures, which can help improve soil aeration and water infiltration.

To successfully rotate wheat and rice, you'll need to consider factors like:

* **Climate:** Both crops have specific climate requirements.
* **Soil type:**  Diff

## Translation

In [16]:
import torch
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)
from IndicTransToolkit import IndicProcessor

# Define model and tokenizer for Indic to English translation
model_name = "ai4bharat/indictrans2-indic-en-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, trust_remote_code=True)

# Initialize the IndicProcessor for preprocessing and postprocessing
ip = IndicProcessor(inference=True)

# Example input sentences in Punjabi (Gurmukhi script)
input_sentences = [
    "ਤੁਸੀਂ ਕਿਵੇਂ ਹੋ?",  # How are you?
    "ਅਸੀਂ ਕੱਲ੍ਹ ਇੱਕ ਨਵੀਂ ਫਿਲਮ ਦੇਖੀ।",  # We watched a new movie yesterday.
    "ਮੇਰੇ ਦੋਸਤ ਨੇ ਮੈਨੂੰ ਉਸਦੇ ਜਨਮਦਿਨ 'ਤੇ ਸੱਦਿਆ ਹੈ।",  # My friend has invited me to his birthday.
    "ਮੈਂ ਹਰ ਰੋਜ਼ ਪਾਰਕ ਜਾਣਦਾ ਹਾਂ।",  # I go to the park every day.
]

# Define source and target languages
src_lang, tgt_lang = "pan_Guru", "eng_Latn"

# Preprocess the input sentences
batch = ip.preprocess_batch(
    input_sentences,
    src_lang=src_lang,
    tgt_lang=tgt_lang,
)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
with tokenizer.as_target_tokenizer():
    generated_tokens = tokenizer.batch_decode(
        generated_tokens.detach().cpu().tolist(),
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True,
    )

# Postprocess the translations
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

# Print the input sentences and their translations
for input_sentence, translation in zip(input_sentences, translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pan_Guru: ਤੁਸੀਂ ਕਿਵੇਂ ਹੋ?
eng_Latn: कैसा हैं?
pan_Guru: ਅਸੀਂ ਕੱਲ੍ਹ ਇੱਕ ਨਵੀਂ ਫਿਲਮ ਦੇਖੀ।
eng_Latn: We saw a new movie yesterday.
pan_Guru: ਮੇਰੇ ਦੋਸਤ ਨੇ ਮੈਨੂੰ ਉਸਦੇ ਜਨਮਦਿਨ 'ਤੇ ਸੱਦਿਆ ਹੈ।
eng_Latn: My friend has invited me for her birthday.
pan_Guru: ਮੈਂ ਹਰ ਰੋਜ਼ ਪਾਰਕ ਜਾਣਦਾ ਹਾਂ।
eng_Latn: I know the park every day.


/Users/pratham/miniconda3/envs/langchainenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [17]:
# Save the model and tokenizer to a directory
model.save_pretrained("indictrans2-indic-en-1B-saved")
tokenizer.save_pretrained("indictrans2-indic-en-1B-saved")

('indictrans2-indic-en-1B-saved/tokenizer_config.json',
 'indictrans2-indic-en-1B-saved/special_tokens_map.json',
 'indictrans2-indic-en-1B-saved/dict.SRC.json',
 'indictrans2-indic-en-1B-saved/dict.TGT.json',
 'indictrans2-indic-en-1B-saved/model.SRC',
 'indictrans2-indic-en-1B-saved/model.TGT',
 'indictrans2-indic-en-1B-saved/added_tokens.json')

In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from IndicTransToolkit import IndicProcessor

def translate_punjabi_to_english(punjabi_sentences):
    """Translates Punjabi sentences to English using the IndicTransToolkit and a pre-trained transformer model.

    Args:
        punjabi_sentences (list): A list of Punjabi sentences to be translated.

    Returns:
        list: A list of corresponding English translations.
    """

    # Load the pre-trained model and tokenizer
    model_name = "ai4bharat/indictrans2-indic-en-1B"
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, trust_remote_code=True)

    # Initialize the IndicProcessor
    ip = IndicProcessor(inference=True)

    # Define source and target languages
    src_lang, tgt_lang = "pan_Guru", "eng_Latn"

    # Preprocess the input sentences
    batch = ip.preprocess_batch(
        punjabi_sentences,
        src_lang=src_lang,
        tgt_lang=tgt_lang,
    )

    # Set the device (GPU or CPU)
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # Tokenize the sentences and generate input encodings
    inputs = tokenizer(
        batch,
        truncation=True,
        padding="longest",
        return_tensors="pt",
        return_attention_mask=True,
    ).to(DEVICE)

    # Generate translations using the model
    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            use_cache=True,
            min_length=0,
            max_length=256,
            num_beams=5,
            num_return_sequences=1,
        )

    # Decode the generated tokens into text
    with tokenizer.as_target_tokenizer():
        generated_tokens = tokenizer.batch_decode(
            generated_tokens.detach().cpu().tolist(),
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

    # Postprocess the translations
    translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

    return translations

In [22]:
punjabi_sentences = [
    "ਤੁਸੀਂ ਕਿਵੇਂ ਹੋ?",  # How are you?
    "ਕਣਕ ਦੀ ਵਰਤੋਂ ਕੀ ਹੈ?",  # We watched a new movie yesterday.
]

In [23]:
english_translations = translate_punjabi_to_english(punjabi_sentences)

/Users/pratham/miniconda3/envs/langchainenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [25]:
english_translations[1]

'What is the use of wheat?'

## punjabi chat

In [27]:
# Function to simulate a continual chat punjabi
def continual_chat():
    print("Start chatting with the AI! Type 'exit' to end the conversation.")
    chat_history = []  # Collect chat history here (a sequence of messages)
    punjabi_sentences=["ਤੁਸੀਂ ਕਿਵੇਂ ਹੋ?"]
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            print("Bye....")
            break
        punjabi_sentences.append(query)
        english_translations = translate_punjabi_to_english(punjabi_sentences)
        new_query=english_translations[-1]
        print(new_query)

        # Process the user's query through the retrieval chain
        result = rag_chain.invoke({"input": new_query, "chat_history": chat_history})
        # Display the AI's response
        print(f"AI: {result['answer']}")
        # Update the chat history
#         chat_history.append(HumanMessage(content=query))
#         chat_history.append(SystemMessage(content=result["answer"]))
        chat_history.append(HumanMessage(content=query))
        chat_history.append(AIMessage(content=result["answer"]))
#         print(chat_history)


# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()


Start chatting with the AI! Type 'exit' to end the conversation.


/Users/pratham/miniconda3/envs/langchainenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


What is the use of wheat?
AI: Wheat is a staple food for many people around the world. It is used to make bread, pasta, noodles, and other food products. It is also used as an ingredient in animal feed. Source: [Wikipedia](https://en.wikipedia.org/wiki/Wheat) 



/Users/pratham/miniconda3/envs/langchainenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


What is food?
AI: Food is any substance consumed to provide nutritional support for the body. It is essential for growth, metabolism, and maintaining life. Food provides the body with energy, nutrients, and building blocks for growth and repair. 



...

Are farmers getting better prices for their produce?
AI: I do not have access to real-time information, including current market prices for produce.  Therefore, I cannot answer whether farmers are getting better prices for their produce. 

To get the most up-to-date information, I recommend checking resources such as:

* **Agricultural news websites:** These websites often report on current market trends and prices.
* **Government agricultural agencies:** Agencies like the USDA in the US provide market data and reports.
* **Local farmer's markets:** Talking to farmers directly can give you insight into their current experiences.

Remember, market prices for produce are constantly fluctuating based on factors like supply, demand, weather conditions, and global events. 



/Users/pratham/miniconda3/envs/langchainenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


What kind of problems do farmers face?
AI: Farmers face a multitude of challenges, some of which are:

* **Weather:** Extreme weather events like droughts, floods, and storms can severely impact crops and livestock.
* **Market Fluctuations:** Prices for agricultural commodities can be volatile, making it difficult to plan and budget.
* **Pests and Diseases:** Pests and diseases can damage crops and livestock, leading to reduced yields and financial losses.
* **Input Costs:** Costs for things like seeds, fertilizer, and fuel are constantly rising, squeezing profit margins.
* **Labor Shortages:** Finding and retaining qualified workers is becoming increasingly difficult, especially in rural areas.
* **Climate Change:** Changing weather patterns and extreme weather events are posing new challenges for farmers.
* **Government Regulations:** Complying with regulations can be complex and costly, adding to the challenges of farming.
* **Competition:** Competition from large-scale agricultural

/Users/pratham/miniconda3/envs/langchainenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


What kind of problems do farmers face?
AI: Farmers face a wide range of challenges, some of which are:

* **Weather:** Extreme weather events like droughts, floods, and storms can severely impact crops and livestock.
* **Market Fluctuations:** Prices for agricultural commodities can be volatile, making it difficult to plan and budget.
* **Pests and Diseases:** Pests and diseases can damage crops and livestock, leading to reduced yields and financial losses.
* **Input Costs:** Costs for things like seeds, fertilizer, and fuel are constantly rising, squeezing profit margins.
* **Labor Shortages:** Finding and retaining qualified workers is becoming increasingly difficult, especially in rural areas.
* **Climate Change:** Changing weather patterns and extreme weather events are posing new challenges for farmers.
* **Government Regulations:** Complying with regulations can be complex and costly, adding to the challenges of farming.
* **Competition:** Competition from large-scale agricultura

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from IndicTransToolkit import IndicProcessor

def translate_punjabi_to_english(punjabi_sentences):
    """Translates Punjabi sentences to English using the IndicTransToolkit and a pre-trained transformer model.

    Args:
        punjabi_sentences (list): A list of Punjabi sentences to be translated.

    Returns:
        list: A list of corresponding English translations.
    """

    # Load the pre-trained model and tokenizer
    model_name = "ai4bharat/indictrans2-indic-en-1B"
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, trust_remote_code=True)

    # Initialize the IndicProcessor
    ip = IndicProcessor(inference=True)

    # Define source and target languages
    src_lang, tgt_lang = "pan_Guru", "eng_Latn"

    # Preprocess the input sentences
    batch = ip.preprocess_batch(
        punjabi_sentences,
        src_lang=src_lang,
        tgt_lang=tgt_lang,
    )

    # Set the device (GPU or CPU)
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # Tokenize the sentences and generate input encodings
    inputs = tokenizer(
        batch,
        truncation=True,
        padding="longest",
        return_tensors="pt",
        return_attention_mask=True,
    ).to(DEVICE)

    # Generate translations using the model
    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            use_cache=True,
            min_length=0,
            max_length=256,
            num_beams=5,
            num_return_sequences=1,
        )

    # Decode the generated tokens into text
    with tokenizer.as_target_tokenizer():
        generated_tokens = tokenizer.batch_decode(
            generated_tokens.detach().cpu().tolist(),
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

    # Postprocess the translations
    translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

    return translations

In [31]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit import IndicProcessor

def translate_english_to_punjabi(english_sentences):
    """Translates English sentences to Punjabi using the provided model and tokenizer.

    Args:
        english_sentences (list): A list of English sentences to be translated.

    Returns:
        list: A list of corresponding Punjabi translations.
    """

    # Load the pre-trained model and tokenizer
    model_name = "ai4bharat/indictrans2-en-indic-1B"
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, trust_remote_code=True)

    # Initialize the IndicProcessor
    ip = IndicProcessor(inference=True)

    # Define source and target languages
    src_lang, tgt_lang = "eng_Latn", "pan_Guru"

    # Preprocess the input sentences
    batch = ip.preprocess_batch(
        english_sentences,
        src_lang=src_lang,
        tgt_lang=tgt_lang,
    )

    # Set the device (GPU or CPU)
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # Tokenize the sentences and generate input encodings
    inputs = tokenizer(
        batch,
        truncation=True,
        padding="longest",
        return_tensors="pt",
        return_attention_mask=True,
    ).to(DEVICE)

    # Generate translations using the model
    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            use_cache=True,
            min_length=0,
            max_length=256,
            num_beams=5,
            num_return_sequences=1,
        )

    # Decode the generated tokens into text
    with tokenizer.as_target_tokenizer():
        generated_tokens = tokenizer.batch_decode(
            generated_tokens.detach().cpu().tolist(),
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

    # Postprocess the translations, including entity replacement
    translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

    return translations

# Example usage
english_sentences = [
    "I love eating pizza.",
    "The weather is beautiful today.",
    "Let's go for a walk in the park."
]

punjabi_translations = translate_english_to_punjabi(english_sentences)

# for english_sentence, punjabi_translation in zip(english_sentences, punjabi_translations):
#     print(f"English: {english_sentence}")
#     print(f"Punjabi: {punjabi_translation}")

In [33]:
# Save the model and tokenizer to a directory
model_name = "indictrans2-en-indic-1B-saved"
model.save_pretrained(model_name)
tokenizer.save_pretrained(model_name)

('indictrans2-en-indic-1B-saved/tokenizer_config.json',
 'indictrans2-en-indic-1B-saved/special_tokens_map.json',
 'indictrans2-en-indic-1B-saved/dict.SRC.json',
 'indictrans2-en-indic-1B-saved/dict.TGT.json',
 'indictrans2-en-indic-1B-saved/model.SRC',
 'indictrans2-en-indic-1B-saved/model.TGT',
 'indictrans2-en-indic-1B-saved/added_tokens.json')

In [32]:
punjabi_translations

['ਮੈਨੂੰ ਪਿੱਜ਼ਾ ਖਾਣਾ ਬਹੁਤ ਪਸੰਦ ਹੈ। ',
 'ਅੱਜ ਮੌਸਮ ਸੁੰਦਰ ਹੈ। ',
 'ਆਓ ਪਾਰਕ ਵਿੱਚ ਸੈਰ ਕਰਨ ਲਈ ਚੱਲੀਏ। ']

## punjabi to punjabi

In [34]:
# Function to simulate a continual chat punjabi to punjabi
def continual_chat():
    print("Start chatting with the AI! Type 'exit' to end the conversation.")
    chat_history = []  # Collect chat history here (a sequence of messages)
    punjabi_sentences=["ਤੁਸੀਂ ਕਿਵੇਂ ਹੋ?"]
    english_sentences=["How are you?"]
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            print("Bye....")
            break
        punjabi_sentences.append(query)
        english_translations = translate_punjabi_to_english(punjabi_sentences)
        new_query=english_translations[-1]
        print(new_query)

        # Process the user's query through the retrieval chain
        result = rag_chain.invoke({"input": new_query, "chat_history": chat_history})
        # Display the AI's response
        print("AI: for english", result["response"])
        english_sentences.append(str(result['answer']))
        punjabi_translations = translate_english_to_punjabi(english_sentences)
        punjabi_response=punjabi_translations[-1]
        print(f"AI: {punjabi_response}")
        # Update the chat history
#         chat_history.append(HumanMessage(content=query))
#         chat_history.append(SystemMessage(content=result["answer"]))
        chat_history.append(HumanMessage(content=query))
        chat_history.append(AIMessage(content=result["answer"]))
#         print(chat_history)


# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()


Start chatting with the AI! Type 'exit' to end the conversation.


/Users/pratham/miniconda3/envs/langchainenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


What kind of problems do farmers face?


/Users/pratham/miniconda3/envs/langchainenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


AI: ਕਿਸਾਨਾਂ ਨੂੰ ਕਈ ਤਰ੍ਹਾਂ ਦੀਆਂ ਚੁਣੌਤੀਆਂ ਦਾ ਸਾਹਮਣਾ ਕਰਨਾ ਪੈਂਦਾ ਹੈ, ਜਿਸ ਵਿੱਚ ਸ਼ਾਮਲ ਹਨਃ * * * ਮੌਸਮਃ * ਸੋਕਾ, ਹਡ਼੍ਹ ਅਤੇ ਗਰਮੀ ਵਰਗੀਆਂ ਮੌਸਮ ਦੀਆਂ ਘਟਨਾਵਾਂ ਫਸਲਾਂ ਦੀ ਪੈਦਾਵਾਰ ਅਤੇ ਪਸ਼ੂਆਂ ਦੀ ਸਿਹਤ ਨੂੰ ਮਹੱਤਵਪੂਰਨ ਤੌਰ'ਤੇ ਪ੍ਰਭਾਵਤ ਕਰ ਸਕਦੀਆਂ ਹਨ। * * ਬਾਜ਼ਾਰ ਦੀ ਅਸਥਿਰਤਾਃ * * ਵਸਤਾਂ ਦੀਆਂ ਕੀਮਤਾਂ ਵਿੱਚ ਉਤਰਾਅ-ਚਡ਼੍ਹਾਅ ਕਿਸਾਨਾਂ ਲਈ ਯੋਜਨਾਬੰਦੀ ਅਤੇ ਮੁਨਾਫਾ ਕਮਾਉਣਾ ਮੁਸ਼ਕਲ ਬਣਾ ਸਕਦੇ ਹਨ। * * * ਨਿਵੇਸ਼ ਲਾਗਤਃ * * ਬੀਜਾਂ, ਖਾਦਾਂ ਅਤੇ ਹੋਰ ਨਿਵੇਸ਼ ਦੀ ਲਾਗਤ ਉੱਚੀ ਅਤੇ ਅਣਹੋਣੀ ਹੋ ਸਕਦੀ ਹੈ। * * * ਮਜ਼ਦੂਰਾਂ ਦੀ ਘਾਟਃ * * ਯੋਗ ਮਜ਼ਦੂਰਾਂ ਨੂੰ ਲੱਭਣਾ ਅਤੇ ਬਰਕਰਾਰ ਰੱਖਣਾ ਇੱਕ ਚੁਣੌਤੀ ਹੋ ਸਕਦੀ ਹੈ, ਖ਼ਾਸਕਰ ਪੀਕ ਸੀਜ਼ਨ ਦੌਰਾਨ। * * * ਨਿਯਮ ਅਤੇ ਨੀਤੀਆਂਃ * * ਕਿਸਾਨਾਂ ਨੂੰ ਨਿਯਮਾਂ ਦੇ ਇੱਕ ਗੁੰਝਲਦਾਰ ਜਾਲ ਦੀ ਪਾਲਣਾ ਕਰਨੀ ਚਾਹੀਦੀ ਹੈ, ਜੋ ਮਹਿੰਗੇ ਅਤੇ ਸਮਾਂ ਲੈਣ ਵਾਲੇ ਹੋ ਸਕਦੇ ਹਨ। * * * ਤਕਨਾਲੋਜੀ ਤੱਕ ਪਹੁੰਚਃ * * ਕਿਸਾਨ ਨਵੀਆਂ ਤਕਨੀਕਾਂ ਨੂੰ ਅਪਣਾਉਣ ਲਈ ਸੰਘਰਸ਼ ਕਰ ਸਕਦੇ ਹਨ ਜੋ ਕੁਸ਼ਲਤਾ ਅਤੇ ਜਲਵਾਯੂ ਪਰਿਵਰਤਨ ਵਿੱਚ ਸੁਧਾਰ ਕਰ ਸਕਦੀਆਂ ਹਨ। 
What schemes is the government implementing for farmers?


KeyboardInterrupt: 

In [36]:
!pip install deep-translator

I0000 00:00:1728943335.764612 1960528 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached deep_translator-1.11.4-py3-none-any.whl.metadata (30 kB)
Using cached deep_translator-1.11.4-py3-none-any.whl (42 kB)


In [ ]:
from deep_translator import GoogleTranslator

# Example usage: English to Punjabi
english_text = "Hello, how are you?"
punjabi_translation = GoogleTranslator(source='en', target='pa').translate(english_text)
print("Punjabi Translation:", punjabi_translation)

# Example usage: Punjabi to English
punjabi_text = "ਕਿਸਾਨਾਂ ਨੂੰ ਕਈ ਤਰ੍ਹਾਂ ਦੀਆਂ ਚੁਣੌਤੀਆਂ ਦਾ ਸਾਹਮਣਾ ਕਰਨਾ ਪੈਂਦਾ ਹੈ, ਜਿਸ ਵਿੱਚ ਸ਼ਾਮਲ ਹਨਃ * * * ਮੌਸਮਃ * ਸੋਕਾ, ਹਡ਼੍ਹ ਅਤੇ ਗਰਮੀ ਵਰਗੀਆਂ ਮੌਸਮ ਦੀਆਂ ਘਟਨਾਵਾਂ ਫਸਲਾਂ ਦੀ ਪੈਦਾਵਾਰ ਅਤੇ ਪਸ਼ੂਆਂ ਦੀ ਸਿਹਤ ਨੂੰ ਮਹੱਤਵਪੂਰਨ ਤੌਰ'ਤੇ ਪ੍ਰਭਾਵਤ ਕਰ ਸਕਦੀਆਂ ਹਨ। * * ਬਾਜ਼ਾਰ ਦੀ ਅਸਥਿਰਤਾਃ * * ਵਸਤਾਂ ਦੀਆਂ ਕੀਮਤਾਂ ਵਿੱਚ ਉਤਰਾਅ-ਚਡ਼੍ਹਾਅ ਕਿਸਾਨਾਂ ਲਈ ਯੋਜਨਾਬੰਦੀ ਅਤੇ ਮੁਨਾਫਾ ਕਮਾਉਣਾ ਮੁਸ਼ਕਲ ਬਣਾ ਸਕਦੇ ਹਨ। * * * ਨਿਵੇਸ਼ ਲਾਗਤਃ * * ਬੀਜਾਂ, ਖਾਦਾਂ ਅਤੇ ਹੋਰ ਨਿਵੇਸ਼ ਦੀ ਲਾਗਤ ਉੱਚੀ ਅਤੇ ਅਣਹੋਣੀ ਹੋ ਸਕਦੀ ਹੈ। * * * ਮਜ਼ਦੂਰਾਂ ਦੀ ਘਾਟਃ * * ਯੋਗ ਮਜ਼ਦੂਰਾਂ ਨੂੰ ਲੱਭਣਾ ਅਤੇ ਬਰਕਰਾਰ ਰੱਖਣਾ ਇੱਕ ਚੁਣੌਤੀ ਹੋ ਸਕਦੀ ਹੈ, ਖ਼ਾਸਕਰ ਪੀਕ ਸੀਜ਼ਨ ਦੌਰਾਨ। * * * ਨਿਯਮ ਅਤੇ ਨੀਤੀਆਂਃ * * ਕਿਸਾਨਾਂ ਨੂੰ ਨਿਯਮਾਂ ਦੇ ਇੱਕ ਗੁੰਝਲਦਾਰ ਜਾਲ ਦੀ ਪਾਲਣਾ ਕਰਨੀ ਚਾਹੀਦੀ ਹੈ, ਜੋ ਮਹਿੰਗੇ ਅਤੇ ਸਮਾਂ ਲੈਣ ਵਾਲੇ ਹੋ ਸਕਦੇ ਹਨ। * * * ਤਕਨਾਲੋਜੀ ਤੱਕ ਪਹੁੰਚਃ * * ਕਿਸਾਨ ਨਵੀਆਂ ਤਕਨੀਕਾਂ ਨੂੰ ਅਪਣਾਉਣ ਲਈ ਸੰਘਰਸ਼ ਕਰ ਸਕਦੇ ਹਨ ਜੋ ਕੁਸ਼ਲਤਾ ਅਤੇ ਜਲਵਾਯੂ ਪਰਿਵਰਤਨ ਵਿੱਚ ਸੁਧਾਰ ਕਰ ਸਕਦੀਆਂ ਹਨ। "
english_translation = GoogleTranslator(source='pa', target='en').translate(punjabi_text)
print("English Translation:", english_translation)


Punjabi Translation: ਹੈਲੋ ਤੁਸੀ ਕਿਵੇਂ ਹੋ?
English Translation: Farmers face a variety of challenges, including * * * Weather * Weather events such as drought, heat and heat can significantly affect crop yields and livestock health. * * Market Volatility * * Fluctuations in commodity prices can make planning and profitability difficult for farmers. * * * Investment Cost * * The cost of seeds, fertilizers and other inputs can be high and unpredictable. * * * Labor shortage * * Finding and retaining qualified workers can be a challenge, especially during peak seasons. * * * Regulations and Policies * * Farmers must comply with a complex web of regulations, which can be costly and time-consuming. * * * Access to Technology * * Farmers may struggle to adopt new technologies that can improve efficiency and climate change.


In [43]:
from deep_translator import GoogleTranslator
def english_to_punjabi(english_text):
    return GoogleTranslator(source='en', target='pa').translate(english_text)

def punjabi_to_english(punjabi_text):
    return GoogleTranslator(source='pa', target='en').translate(punjabi_text)


In [66]:
# english_to_punjabi("""In a quiet village nestled between rolling hills, there lived a curious girl named Lily. With her wild curls and bright green eyes, she had a knack for finding adventure in the most ordinary places. One sunny afternoon, while exploring the outskirts of her village, Lily stumbled upon an old, wrought-iron gate hidden behind a curtain of ivy.

# Intrigued, she pushed the gate open, its rusty hinges creaking as if awakening from a long slumber. Beyond the gate lay a path overgrown with wildflowers, leading into a secluded garden. The air was thick with the sweet scent of blooming jasmine and honeysuckle, and the gentle hum of bees filled her ears. As she ventured further, she marveled at the vibrant colors and sounds around her.

# In the heart of the garden stood a majestic tree, its branches stretching wide like welcoming arms. Beneath its shade, a weathered stone bench invited her to sit and rest. Lily approached the bench and noticed an engraved plaque that read, “To those who seek, beauty awaits.” Puzzled but delighted, she took a seat, allowing the tranquility of the garden to wash over her.

# As she sat there, a flutter of movement caught her eye. A delicate butterfly, with wings painted in hues of blue and gold, danced around her. Entranced, Lily followed the butterfly as it led her deeper into the garden. Each step revealed more wonders: a sparkling fountain, hidden among the ferns, and a mosaic of colorful pebbles forming a path to a secret nook.

# In this nook, she discovered a group of children playing, their laughter ringing out like music. They were painting stones, their hands smeared with vibrant colors. One of the children, a boy with bright red hair and a contagious smile, waved her over. “Join us! We’re creating magic!” he exclaimed.

# Lily hesitated for a moment, then approached the group. They welcomed her warmly, and soon she was painting stones with them, her imagination igniting as she transformed ordinary rocks into whimsical creatures and dazzling patterns. Time seemed to slip away as they shared stories and laughter, forging bonds that felt timeless.

# As the sun began to dip below the horizon, casting a golden glow over the garden, Lily realized she needed to return home. The children understood her need to leave but invited her to return. “This garden is a place of magic,” the boy with red hair said. “Whenever you feel lost, just follow the path of wildflowers.”

# With a heart full of joy and a pocket full of painted stones, Lily made her way back to the iron gate. She paused to take one last look at the hidden garden, a world of wonder and friendship that felt like a dream. As she stepped through the gate and closed it behind her, she felt a sense of promise—a promise of return.

# From that day on, the garden became her sanctuary. Whenever life felt overwhelming or mundane, she would escape to her secret place, paint stones with her new friends, and bask in the beauty of the hidden garden. With each visit, she learned that magic existed not just in the garden but within herself and the connections she made along the way.

# Lily’s adventures in the garden taught her that beauty and friendship often lie just beyond the ordinary, waiting for those brave enough to seek them out. And so, with a heart full of hope and a spirit of adventure, she continued to explore the world, one hidden garden at a time.""")

In [45]:
# punjabi_to_english("""ਇੱਕ ਵਾਰੀ ਦੀ ਗੱਲ ਹੈ, ਪਿੰਡ ਦੇ ਇੱਕ ਛੋਟੇ ਘਰ ਵਿਚ ਇੱਕ ਕੁੜੀ ਰਹਿੰਦੀ ਸੀ ਜਿਸਦਾ ਨਾਮ ਸੀ ਸਿਮਰਨ। ਸਿਮਰਨ ਬਹੁਤ ਸੁੰਦਰ ਅਤੇ ਮਿਹਨਤੀ ਸੀ। ਉਹ ਹਰ ਰੋਜ ਆਪਣੇ ਪਿੰਡ ਦੇ ਖੇਤਾਂ ਵਿਚ ਕੰਮ ਕਰਦੀ ਅਤੇ ਦੁਪਹਿਰ ਨੂੰ ਵਾਪਸ ਆ ਕੇ ਆਪਣੇ ਦਾਦੀ-ਨਾਨੀ ਦੀਆਂ ਗੱਲਾਂ ਸੁਣਦੀ। ਉਸ ਦੀ ਦਾਦੀ ਹਮੇਸ਼ਾ ਉਸਨੂੰ ਪੁਰਾਣੀਆਂ ਕਹਾਣੀਆਂ ਸੁਣਾਉਂਦੀ ਸੀ, ਜੋ ਕਿ ਉਸਦੀ ਮਾਂ ਦੀਆਂ ਕਹਾਣੀਆਂ ਨਾਲ ਮਿਲਦੀਆਂ ਜੁਲਦੀਆਂ ਹੁੰਦੀਆਂ ਸਨ।

# ਇੱਕ ਦਿਨ, ਸਿਮਰਨ ਨੇ ਆਪਣੀ ਦਾਦੀ ਤੋਂ ਸੁਣਿਆ ਕਿ ਪਿੰਡ ਦੇ ਬਾਹਰ ਇੱਕ ਜਾਦੂਈ ਜੰਗਲ ਹੈ, ਜਿਸ ਵਿੱਚ ਸਭ ਕੁਝ ਸੰਭਵ ਹੈ। ਜੰਗਲ ਦੇ ਵਿਚਕਾਰ ਇੱਕ ਜਾਦੂਈ ਝੀਲ ਹੈ, ਜੋ ਲੋਕਾਂ ਦੀਆਂ ਦੁਖਦਾਈਆਂ ਇੱਛਾਵਾਂ ਨੂੰ ਪੂਰਾ ਕਰਦੀ ਹੈ। ਸਿਮਰਨ ਦੇ ਦਿਲ ਵਿਚ ਜਿਗਿਆਸਾ ਜਾਗੀ। ਉਸਨੇ ਸੋਚਿਆ ਕਿ ਉਹ ਇਸ ਜੰਗਲ ਨੂੰ ਖੋਜਣਾ ਚਾਹੁੰਦੀ ਹੈ।

# ਸਿਮਰਨ ਨੇ ਦਿਨ ਚੁਣਿਆ ਅਤੇ ਸਵੇਰੇ ਸਵੇਰੇ ਨਿਕਲੀ। ਜੰਗਲ ਵਿਚ ਚਲਦੇ-ਚਲਦੇ ਉਸਨੇ ਸੁੰਦਰ ਪੱਖੀਆਂ, ਰੰਗੀਨ ਫੁੱਲ ਅਤੇ ਉੱਚੀਆਂ ਦਰਖਤਾਂ ਨੂੰ ਦੇਖਿਆ। ਉਸ ਨੂੰ ਸਮਝ ਆ ਗਿਆ ਕਿ ਇਹ ਸੱਚਮੁਚ ਇੱਕ ਜਾਦੂਈ ਜੰਗਲ ਹੈ। ਜਦੋਂ ਉਹ ਝੀਲ ਦੇ ਨੇੜੇ ਪਹੁੰਚੀ, ਉਸਨੇ ਵੇਖਿਆ ਕਿ ਝੀਲ ਦਾ ਪਾਣੀ ਚਮਕਦਾਰ ਹੈ ਅਤੇ ਇਸ ਵਿਚ ਕੁਝ ਵਿਲੱਖਣ ਹਨ। ਉਹ ਝੀਲ ਦੇ ਪਾਣੀ ਵਿਚ ਆਪਣੇ ਪੈਰ ਮਾਰ ਕੇ ਪਾਣੀ ਨੂੰ ਛੂਹਦੀ ਹੈ, ਅਤੇ ਉਸ ਦੀਆਂ ਇੱਛਾਵਾਂ ਦਾ ਆਰੰਭ ਹੁੰਦਾ ਹੈ।

# ਸਿਮਰਨ ਨੇ ਆਪਣੀ ਪਹਿਲੀ ਇੱਛਾ ਮੰਗੀ: "ਮੈਂ ਚਾਹੁੰਦੀ ਹਾਂ ਕਿ ਮੇਰੇ ਪਿੰਡ ਦੇ ਲੋਕ ਖੁਸ਼ ਰਹਿਣ।" ਝੀਲ ਦੇ ਪਾਣੀ ਨੇ ਚਮਕਣਾ ਸ਼ੁਰੂ ਕਰ ਦਿੱਤਾ, ਅਤੇ ਸਿਮਰਨ ਨੂੰ ਅਹਿਸਾਸ ਹੋਇਆ ਕਿ ਇਹ ਜਾਦੂ ਹੋ ਰਿਹਾ ਹੈ। ਅਗਲੇ ਪਲ, ਉਸ ਦੇ ਪਿੰਡ ਦੇ ਲੋਕਾਂ ਨੂੰ ਹੱਸਦੇ-ਖੇਡਦੇ ਦੇਖ ਕੇ ਉਸ ਦਾ ਦਿਲ ਖੁਸ਼ ਹੋ ਗਿਆ।

# ਫਿਰ, ਉਸਨੇ ਦੂਜੀ ਇੱਛਾ ਮੰਗੀ: "ਮੈਨੂੰ ਸਿੱਖਿਆ ਦੇਣ ਵਾਲਾ ਇੱਕ ਵਿਦਿਆਰਥੀ ਮਿਲ ਜਾਵੇ!" ਝੀਲ ਨੇ ਇੱਕ ਸੁੰਦਰ ਕਿਤਾਬ ਉਸਦੇ ਸਾਹਮਣੇ ਪੇਸ਼ ਕੀਤੀ। ਉਸਨੇ ਉਸ ਕਿਤਾਬ ਨੂੰ ਖੋਲ੍ਹਿਆ, ਅਤੇ ਉਸ ਵਿਚ ਬਹੁਤ ਸਾਰੇ ਗਿਆਨ ਅਤੇ ਸਿੱਖਿਆ ਮਿਲੀ। ਸਿਮਰਨ ਨੇ ਉਸ ਕਿਤਾਬ ਦੇ ਸਹਾਰੇ ਆਪਣੇ ਪਿੰਡ ਦੇ ਬੱਚਿਆਂ ਨੂੰ ਪੜ੍ਹਨ ਲਈ ਪ੍ਰੇਰਿਤ ਕੀਤਾ।

# ਆਖਿਰ ਵਿਚ, ਸਿਮਰਨ ਨੇ ਤੀਜੀ ਇੱਛਾ ਮੰਗੀ: "ਮੈਨੂੰ ਦਾਦੀ ਦੇ ਸਾਥ ਬਿਟਾਉਣ ਲਈ ਸਮਾਂ ਮਿਲੇ!" ਝੀਲ ਦੇ ਪਾਣੀ ਨੇ ਉਸ ਨੂੰ ਇਹ ਚਾਹੁਣਾ ਵੀ ਪੂਰਾ ਕੀਤਾ। ਸਿਮਰਨ ਦੇ ਦਾਦੀ ਨੇ ਉਸਨੂੰ ਬਹੁਤ ਪਿਆਰ ਨਾਲ ਝੀਲ ਦੇ ਬਾਰੇ ਦੱਸਿਆ, ਅਤੇ ਦੋਹਾਂ ਨੇ ਮਿਲ ਕੇ ਆਪਣੀਆਂ ਯਾਦਾਂ ਨੂੰ ਸਾਂਝਾ ਕੀਤਾ।

# ਸਿਮਰਨ ਨੇ ਅਖੀਰ ਦੇ ਵਿੱਚ ਸੋਚਿਆ ਕਿ ਜਾਦੂ ਦਾ ਸੱਚਾ ਪ੍ਰਭਾਵ ਲੋਕਾਂ ਦੇ ਸਹਾਰੇ ਅਤੇ ਪ੍ਰੇਮ ਵਿੱਚ ਹੈ। ਉਸਨੇ ਇਹੀ ਸਿੱਖਿਆ ਆਪਣੇ ਪਿੰਡ ਦੇ ਲੋਕਾਂ ਨਾਲ ਸਾਂਝੀ ਕੀਤੀ, ਅਤੇ ਸਾਰੇ ਪਿੰਡ ਨੇ ਮਿਲ ਕੇ ਖੁਸ਼ੀਆਂ ਮਨਾਉਣ ਦੀ ਯੋਜਨਾ ਬਣਾਈ। ਇਹ ਜਾਦੂਈ ਯਾਤਰਾ ਸਿਮਰਨ ਲਈ ਇੱਕ ਸਿਖਰ ਹੋ ਗਈ, ਅਤੇ ਉਸਨੇ ਸਿੱਖਿਆ ਕਿ ਸੱਚੀ ਖੁਸ਼ੀ ਦੋਸ਼ੀਆਂ, ਦੋਸਤਾਂ ਅਤੇ ਪ੍ਰੇਮ ਨਾਲ ਹੀ ਮਿਲਦੀ ਹੈ।

# ਉਸ ਦੀ ਜਾਦੂਈ ਯਾਤਰਾ ਨੇ ਸਿਮਰਨ ਨੂੰ ਸਮਝਾਇਆ ਕਿ ਜਦੋਂ ਅਸੀਂ ਕਿਸੇ ਨੂੰ ਖੁਸ਼ੀ ਦਿੰਦੇ ਹਾਂ, ਤਾਂ ਸਾਡੇ ਆਪਣੇ ਦਿਲ ਵਿਚ ਵੀ ਖੁਸ਼ੀ ਹੁੰਦੀ ਹੈ।""")

'Once upon a time, there lived a girl named Simran in a small house in the village. Simran was very beautiful and laborious. She used to work in the fields of her village every day and would come back in the afternoon and listen to the stories of her grandparents. His grandmother always told him old stories, which were similar to his mother\'s stories.\n\nOne day, Simran hears from her grandmother that there is a magical forest outside the village, where everything is possible. In the middle of the forest is a magical lake, which fulfills people\'s tragic wishes. Curiosity awakened in Simran\'s heart. She thought she wanted to explore this forest.\n\nSimran chose the day and left early in the morning. Walking through the forest, he saw beautiful birds, colorful flowers and tall trees. He realized that it was indeed an enchanted forest. As she approached the lake, she saw that the water of the lake was bright and had something unique in it. She dips her feet into the water of the lake a

## chatbot translate google

In [53]:
# Define the persistent directory
# current_dir = os.path.dirname(os.path.abspath(__file__))
# persistent_directory = os.path.join(current_dir, "db", "chroma_db_with_metadata")
persistent_directory = ".db/chroma_db_with_metadata"

# Define the embedding model
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory, embedding_function=huggingface_embeddings)

# Create a retriever for querying the vector store
# `search_type` specifies the type of search (e.g., similarity)
# `search_kwargs` contains additional arguments for the search (e.g., number of results to return)
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

# Create a google model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Contextualize question prompt
# This system prompt helps the AI understand that it should reformulate the question
# based on the chat history to make it a standalone question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

# Create a prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a history-aware retriever
# This uses the LLM to help reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Answer question prompt
# This system prompt helps the AI understand that it should provide concise answers
# based on the retrieved context and indicates what to do if the answer is unknown
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use optimal number of sentences to answer the question. "
    "Provide the source as well. "
    "\n\n"
    "{context}"
)

# Create a prompt template for answering questions
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a chain to combine documents for question answering
# `create_stuff_documents_chain` feeds all retrieved context into the LLM
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create a retrieval chain that combines the history-aware retriever and the question answering chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# Function to simulate a continual chat punjabi to punjabi
def continual_chat():
    print("Start chatting with the AI! Type 'exit' to end the conversation.")
    chat_history = []  # Collect chat history here (a sequence of messages)

    while True:
        query = input("You: ")
        if query.lower() == "exit":
            print("Bye....")
            break

        new_query = punjabi_to_english(str(query))
        
        print(new_query)

        # Process the user's query through the retrieval chain
        result = rag_chain.invoke({"input": new_query, "chat_history": chat_history})
        # Display the AI's response
        print("AI: for english", result['answer'])
        punjabi_response = english_to_punjabi(str(result['answer']))
    
        print(f"AI: {punjabi_response}")

        # Update the chat history
#         chat_history.append(HumanMessage(content=query))
#         chat_history.append(SystemMessage(content=result["answer"]))
        chat_history.append(HumanMessage(content=query))
        chat_history.append(AIMessage(content=result["answer"]))
#         print(chat_history)


# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()


Start chatting with the AI! Type 'exit' to end the conversation.
What are pesticides, and their uses?
AI: for english Pesticides are substances that are used to kill or control pests, which can include insects, weeds, fungi, bacteria, viruses, and rodents. They are used in agriculture, forestry, public health, and other areas to protect crops, livestock, and humans from pests. Pesticides can be applied in a variety of ways, including spraying, dusting, and fumigation. They can be made from synthetic chemicals, natural products, or a combination of both.  

Pesticides are used for a variety of purposes, including:

* **Protecting crops from pests:** Pesticides are used to control insects, weeds, and diseases that can damage crops and reduce yields.
* **Controlling pests in homes and gardens:** Pesticides are used to kill insects, rodents, and other pests that can infest homes and gardens.
* **Protecting livestock from pests:** Pesticides are used to control insects, ticks, and other pes

In [57]:
!pip install --upgrade httpx httpcore
!pip uninstall googletrans -y
!pip install googletrans==4.0.0-rc1

I0000 00:00:1728945266.303337 1960528 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


I0000 00:00:1728945268.993585 1960528 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: googletrans 4.0.0rc1
Uninstalling googletrans-4.0.0rc1:
  Successfully uninstalled googletrans-4.0.0rc1


I0000 00:00:1728945270.755544 1960528 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached googletrans-4.0.0rc1-py3-none-any.whl
  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.6
    Uninstalling httpcore-1.0.6:
      Successfully uninstalled httpcore-1.0.6
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.2
    Uninstalling httpx-0.27.2:
      Successfully uninstalled httpx-0.27.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol

In [62]:
!pip install langdetect


I0000 00:00:1728945408.864610 1960528 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=95dec7dc7f61aa7ea8752d353e62fc3cb82d84e4af3f96342dbc73a32177913e
  Stored in directory: /Users/pratham/Library/Caches/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [67]:
from langdetect import detect, DetectorFactory

# Ensure consistent results
DetectorFactory.seed = 0

def detect_language(text):
    try:
        # Detect the language of the input text
        language_code = detect(text)
        return language_code
    except Exception as e:
        return str(e)

# Test the function with Punjabi text
punjabi_text = "ਤੁਸੀਂ ਕਿਵੇਂ ਹੋ?"
detected_language = detect_language(punjabi_text)
print(f"The detected language code is: {detected_language}")

# You can also test with English
english_text = "How are you?"
detected_language = detect_language(english_text)
print(f"The detected language code is: {detected_language}")

# Test with Hindi
hindi_text = "आप कैसे हैं?"
detected_language = detect_language(hindi_text)
print(f"The detected language code is: {detected_language}")


The detected language code is: pa
The detected language code is: en
The detected language code is: hi


## detect language of the prompt

In [73]:
# Define the persistent directory
# current_dir = os.path.dirname(os.path.abspath(__file__))
# persistent_directory = os.path.join(current_dir, "db", "chroma_db_with_metadata")
# persistent_directory = ".db/chroma_db_with_metadata"
persistent_directory = "db/chroma_db_with_metadata"

# Define the embedding model
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory, embedding_function=huggingface_embeddings)

# Create a retriever for querying the vector store
# `search_type` specifies the type of search (e.g., similarity)
# `search_kwargs` contains additional arguments for the search (e.g., number of results to return)
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

# Create a google model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Contextualize question prompt
# This system prompt helps the AI understand that it should reformulate the question
# based on the chat history to make it a standalone question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

# Create a prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a history-aware retriever
# This uses the LLM to help reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Answer question prompt
# This system prompt helps the AI understand that it should provide concise answers
# based on the retrieved context and indicates what to do if the answer is unknown
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use optimal number of sentences to answer the question. "
    "Provide the source as well. "
    "\n\n"
    "{context}"
)

# Create a prompt template for answering questions
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a chain to combine documents for question answering
# `create_stuff_documents_chain` feeds all retrieved context into the LLM
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create a retrieval chain that combines the history-aware retriever and the question answering chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# Function to simulate a continual chat punjabi to punjabi
def continual_chat():
    print("Start chatting with the AI! Type 'exit' to end the conversation.")
    chat_history = []  # Collect chat history here (a sequence of messages)

    while True:
        query = input("You: ")
        if query.lower() == "exit":
            print("Bye....")
            break

        language=detect_language(query)
        
        if language.lower()=="pa":
            query = punjabi_to_english(str(query))
        
            print(query)
        else:
            print(query)

        # Process the user's query through the retrieval chain
        result = rag_chain.invoke({"input": query, "chat_history": chat_history})
        # Display the AI's response
        
        if language.lower()=="pa":
            punjabi_response = english_to_punjabi(str(result['answer']))
    
            print(f"AI: {punjabi_response}")
        else:
            print("AI:", result['answer'])


        # Update the chat history
#         chat_history.append(HumanMessage(content=query))
#         chat_history.append(SystemMessage(content=result["answer"]))
        chat_history.append(HumanMessage(content=query))
        chat_history.append(AIMessage(content=result["answer"]))
#         print(chat_history)


# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()


Start chatting with the AI! Type 'exit' to end the conversation.
what is rice?
AI: Rice is a type of grain that is a staple food for many people around the world. It is a grass species originally domesticated in the Asian subcontinent and is now cultivated in many parts of the world.  It is the most widely consumed grain in the world.  It is a good source of carbohydrates and is often eaten with other foods, such as vegetables, meat, and fish.

Source:  Farmers Handbook on Basic Agriculture 

who is joginder singh?
AI: Joginder Singh is a professor from Punjab Agricultural University, mentioned in the provided text as having offered valuable suggestions for the study. 

please tell me more about him.
AI: Unfortunately, the provided text only mentions Joginder Singh as a professor from Punjab Agricultural University who contributed to the study. There is no further information about him, such as his specific area of expertise or any publications. 

To learn more about Joginder Singh, yo

## More data


In [76]:
db_dir = os.path.join('./', "db")
persistent_directory = os.path.join(db_dir, "extended_chroma_db_with_metadata")

In [77]:
create_vector_database(huggingface_embeddings,persistent_directory)

Persistent directory does not exist. Initializing vector store...

--- Document Chunks Information ---
Number of document chunks: 33

--- Creating embeddings ---

--- Finished creating embeddings ---

--- Creating and persisting vector store ---

--- Finished creating and persisting vector store ---


In [80]:
# Define the persistent directory
# current_dir = os.path.dirname(os.path.abspath(__file__))
# persistent_directory = os.path.join(current_dir, "db", "chroma_db_with_metadata")
# persistent_directory = ".db/chroma_db_with_metadata"
persistent_directory = "db/extended_chroma_db_with_metadata"

# Define the embedding model
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory, embedding_function=huggingface_embeddings)

# Create a retriever for querying the vector store
# `search_type` specifies the type of search (e.g., similarity)
# `search_kwargs` contains additional arguments for the search (e.g., number of results to return)
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

# Create a google model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Contextualize question prompt
# This system prompt helps the AI understand that it should reformulate the question
# based on the chat history to make it a standalone question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

# Create a prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a history-aware retriever
# This uses the LLM to help reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Answer question prompt
# This system prompt helps the AI understand that it should provide concise answers
# based on the retrieved context and indicates what to do if the answer is unknown
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use optimal number of sentences to answer the question. "
    "Provide the source as well. "
    "\n\n"
    "{context}"
)

# Create a prompt template for answering questions
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a chain to combine documents for question answering
# `create_stuff_documents_chain` feeds all retrieved context into the LLM
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create a retrieval chain that combines the history-aware retriever and the question answering chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# Function to simulate a continual chat punjabi to punjabi
def continual_chat():
    print("Start chatting with the AI! Type 'exit' to end the conversation.")
    chat_history = []  # Collect chat history here (a sequence of messages)

    while True:
        query = input("You: ")
        if query.lower() == "exit":
            print("Bye....")
            break

        language=detect_language(query)
        
        if language.lower()=="pa":
            query = punjabi_to_english(str(query))
        
            print(query)
        else:
            print(query)

        # Process the user's query through the retrieval chain
        result = rag_chain.invoke({"input": query, "chat_history": chat_history})
        # Display the AI's response
        
        if language.lower()=="pa":
            punjabi_response = english_to_punjabi(str(result['answer']))
    
            print(f"AI: {punjabi_response}")
        else:
            print("AI:", result['answer'])


        # Update the chat history
#         chat_history.append(HumanMessage(content=query))
#         chat_history.append(SystemMessage(content=result["answer"]))
        chat_history.append(HumanMessage(content=query))
        chat_history.append(AIMessage(content=result["answer"]))
#         print(chat_history)


# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()


Start chatting with the AI! Type 'exit' to end the conversation.
what topics can you infer from the text?
AI: The text discusses the state of agriculture in Punjab, India, focusing on the changing trends and challenges the state's agricultural sector faces. Here are some key topics:

* **Agricultural Stagnation:** The text explores the reasons behind the stagnation of Punjab's agricultural sector, which was once considered the "breadbasket of India". 
* **Green Revolution:** The text examines the impact of the Green Revolution on Punjab's agriculture, both positive and negative, and how its effects have led to the current challenges.
* **Ecological Degradation:**  The text highlights the environmental consequences of Punjab's agricultural practices, including groundwater depletion, soil degradation, and air pollution from paddy straw burning.
* **Socioeconomic Issues:** The text examines the social and economic consequences of agricultural stagnation, including declining farmer incomes